# Create dataframes for triples

In [22]:
import pandas as pd
from tqdm.notebook import tqdm
import os
from sqlalchemy import create_engine

engine = create_engine('sqlite:///Download/main.db')

PATH = "Download/Cleaned Data/"

for file in os.listdir(PATH):
    
    df = pd.read_csv(os.path.join(PATH,file))

    df.to_sql(os.path.splitext(file)[0], con=engine, if_exists='replace')

In [24]:
engine.table_names()

['Confession',
 'Confessions',
 'CreepyPasta',
 'NoSleep',
 'Self',
 'ShortScaryStories',
 'SpanishText',
 'StoriesAboutKevin',
 'TIFU']

In [27]:
df = pd.read_sql_table('NoSleep', 'sqlite:///Download/main.db')  
# df['selftext'].to_list()
df

,index,created_utc,id,score,selftext,subreddit,title,title_language,selftext_language
0,0,1269397810,bhef5,24,Hey guys! Thanks for checking out /r/NoSleep! ...,nosleep,Welcome to /r/NoSleep!,nl,en
1,1,1269458850,bhq4w,68,This is copypasta from an earlier post... no w...,nosleep,The only possible supernatural experience I've...,en,en
2,2,1269476233,bhtlo,19,Reposting at the request of another. This was ...,nosleep,The Longhorn,en,en
3,3,1269479929,bhu80,29,"So, after looking at some other subreddits, I ...",nosleep,/r/NoSleep Rules,af,en
4,4,1269481302,bhufy,18,This is copy and pasted from the thread that s...,nosleep,My dad's story,cy,en
...,...,...,...,...,...,...,...,...,...
173274,173274,1591452441,gxrm8v,1,"""Any sign of 'em yet?"" \n\nI continued staring...",nosleep,I'm tasked with killing nameless things out in...,en,en
173275,173275,1591452568,gxrnj7,1,They say the devil is in the details. Well th...,nosleep,"I picked up a hitchhiker by mistake, now he's ...",en,en
173276,173276,1591453707,gxrytp,1,It is hard for me to talk about my old friend ...,nosleep,My Best Friend Saw Bugs Under His Skin,de,en
173277,173277,1591454493,gxs6jf,1,This is the only rule of our household. If you...,nosleep,Do NOT open your eyes. (The Beginning),en,en


In [30]:
import spacy
nlp = spacy.load('en_core_web_lg')

from openie import StanfordOpenIE
import truecase
from tqdm.notebook import tqdm

with StanfordOpenIE() as client:
    for table in tqdm(engine.table_names()):
        df = pd.read_sql_table(table, 'sqlite:///Download/main.db').head(1000)
        df_li = []
        for story_id, story in tqdm(zip(df['id'].to_list(), df['selftext'].to_list()), total = len(df['id'].to_list())):
            doc = nlp(story)
            for sent in doc.sents:
                original_text = sent.text
                truecase_text = truecase.get_true_case(original_text)
                text = truecase_text[0].lower() + truecase_text[1:]
                for triple in client.annotate(text):
                    triple['id'] = story_id
                    triple['subreddit'] = table
                    triple['original sent'] = text
                    df_li.append(triple)
            
        kg_df = pd.DataFrame(df_li)
        kg_df.to_sql(f"{table}_kg", con=engine, if_exists='replace')

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Starting server with command: java -Xmx8G -cp /home/anthony/stanfordnlp_resources/stanford-corenlp-full-2018-10-05/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-b7156e21bcdc44a4.props -preload openie


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/144 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [34]:
engine.table_names()

['Confession',
 'Confession_kg',
 'Confessions',
 'Confessions_kg',
 'CreepyPasta',
 'CreepyPasta_kg',
 'NoSleep',
 'NoSleep_kg',
 'Self',
 'Self_kg',
 'ShortScaryStories',
 'ShortScaryStories_kg',
 'SpanishText',
 'SpanishText_kg',
 'StoriesAboutKevin',
 'StoriesAboutKevin_kg',
 'TIFU',
 'TIFU_kg']

In [45]:
print(engine.execute("SELECT * FROM NoSleep LIMIT 10").fetchall()[4][4])

This is copy and pasted from the thread that started this subreddit, it was buried in the other thread so I figured I'd repost it here where some people can see it.

This is absolutely true, and while it doesn't meld with my beliefs, I believe him. My dad told this story to my mom and no one else, I asked him later to verify his "ghost story", he told me it really happened, but didn't want to talk about it. For some background, I'm 99 percent sure my dad is an atheist, he's a very logical and skeptical person, and only believes in ghosts because he's seen them (according to him).

He works for the airlines, and as part of his job, he's always traveling and staying in small hotels near the airports he services. I think he was in St Louis when this happened. He was laying in bed, and had just turned the tv off, the door was locked. He wasn't falling asleep yet, but was getting ready to. It was very cold and rainy outside, so he could hear the rain falling. When he was laying in bed he he

In [36]:
import requests
obj = requests.get('http://api.conceptnet.io/c/en/apple').json()

In [37]:
obj.keys()

dict_keys(['@context', '@id', 'edges', 'version', 'view'])

In [38]:
obj

{'@context': ['http://api.conceptnet.io/ld/conceptnet5.7/context.ld.json'],
 '@id': '/c/en/apple',
 'edges': [{'@id': '/a/[/r/RelatedTo/,/c/en/apple/,/c/en/fruit/]',
   '@type': 'Edge',
   'dataset': '/d/verbosity',
   'end': {'@id': '/c/en/fruit',
    '@type': 'Node',
    'label': 'fruit',
    'language': 'en',
    'term': '/c/en/fruit'},
   'license': 'cc:by/4.0',
   'rel': {'@id': '/r/RelatedTo', '@type': 'Relation', 'label': 'RelatedTo'},
   'sources': [{'@id': '/and/[/s/process/split_words/,/s/resource/verbosity/]',
     '@type': 'Source',
     'contributor': '/s/resource/verbosity',
     'process': '/s/process/split_words'},
    {'@id': '/s/resource/verbosity',
     '@type': 'Source',
     'contributor': '/s/resource/verbosity'}],
   'start': {'@id': '/c/en/apple',
    '@type': 'Node',
    'label': 'apple',
    'language': 'en',
    'term': '/c/en/apple'},
   'surfaceText': '[[apple]] is related to [[fruit]]',
   'weight': 12.80968383684781},
  {'@id': '/a/[/r/HasProperty/,/c/en/